# SIMPLEFEM - A simple implementation of a finite elements method


This is a simple implementation of a finite element method based on the MATLAB code of [Prof. Jörn Behrens](https://www.math.uni-hamburg.de/forschung/bereiche/am/numgeo/personen/behrens-joern.html) (University of Hamburg) in his lecture *Numerical Approximation of Partial Differential Equations - Galerkin Methods*.

Set up mesh with 9 nodes and and 8 elements in the variables `coord`, `elem`, and `emax`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.linalg import spsolve
import scipy.sparse as sp
import time

def initial():
  """
  Sets up the initial triangulation on the
  unit square with 8 meshes and 9 nodes.

  Input:
    None

  Output:
    coord   coordinates of nodes in mesh
    elem    element nodes indices
    emax    start and end indicies of triangulation levels
  """
  global coord, elem, emax
  coord = [
        [0., 0.],  [0.5, 0.],  [1., 0.],
        [0., 0.5], [0.5, 0.5], [1., 0.5],
        [0., 1.],  [0.5, 1.],  [1., 1.]
            ]

  elem = [
        [0, 4, 3], [4, 0, 1], [1, 5, 4], [5, 1, 2],
        [3, 7, 6], [7, 3, 4], [4, 8, 7], [8, 4, 5]
          ]

  emax = [
      [0, 7]
      ]

  print("Initial mesh with 8 elements and 9 nodes initialized.")

initial()

Plot mesh if you want to inspect

In [ ]:
def plot_mesh(lvl=-1, show_elem_ids=True, show_node_ids=False):
  """
  Plots the mesh of the triangulation.

  Input:
    lvl       which level to plot, default is the most dense level
    elem_ids  whether plot the ids of the elements in their center
    node_ids  whether plot the ids of the nodes next to them

  Output:
    None
  """
  _, ax = plt.subplots()

  # Get level bounds
  lmn, lmx = emax[lvl]

  # Go through every element
  for i, e in enumerate(elem[lmn:lmx+1], start=lmn):
    # Format: e = [node1, node2, node3] and coord[e[i]] = [xi, yi]

    # Get node coordinates
    x0, y0 = coord[e[0]]
    x1, y1 = coord[e[1]]
    x2, y2 = coord[e[2]]
    x = [x0, x1, x2]
    linelength=np.max(x)-np.min(x)

    # Plot nodes and lines between them
    ax.plot( [x0, x1], [y0, y1], color="black", marker='o')
    ax.plot( [x0, x2], [y0, y2], color="black", marker='o')
    ax.plot( [x1, x2], [y1, y2], color="black", marker='o')

    if show_elem_ids:
      # Plot element numbers in the middle of element
      x_mean = (x0+x1+x2)/3
      y_mean = (y0+y1+y2)/3
      ax.text(x_mean, y_mean, f"{i+1}", color="grey", fontsize=52*linelength)

    if show_node_ids:
      # Plot nodes numbers
      pad = [0.01, -0.04]
      ax.text(x0+pad[0], y0+pad[1], f"{e[0]}", fontsize=32*linelength)
      ax.text(x1+pad[0], y1+pad[1], f"{e[1]}", fontsize=32*linelength)
      ax.text(x2+pad[0], y2+pad[1], f"{e[2]}", fontsize=32*linelength)

  plt.show()

plot_mesh(show_node_ids=True)

Refine the mesh for `lvl` times

In [ ]:
def refine(lvl):
  """
  Refines the mesh lvl times. Every refinement doubles the elements.

  Input:
    lvl     number of refinements to perform

  Output:
    coord   extended coordinates array
    elem    refined mesh
  """
  global t_refine
  t_refine = time.time()

  def elemrefine(ie):
    ### Create 3 new nodes
    # Get node ids
    e = elem[ie]

    # New nodes in the middle of existing of nodes
    n1 = ( 0.5 * ( np.array(coord[e[0]]) + np.array(coord[e[1]]) ) ).tolist()
    n2 = ( 0.5 * ( np.array(coord[e[1]]) + np.array(coord[e[2]]) ) ).tolist()
    n3 = ( 0.5 * ( np.array(coord[e[0]]) + np.array(coord[e[2]]) ) ).tolist()

    # Check if nodes already exist
    i1 = -1
    nodes_ind = len(coord)-1
    for i, node in enumerate(coord):
      if node[0] == n1[0] and node[1] == n1[1]:
        i1 = i
        break
    else: # Node does not exist
      nodes_ind += 1
      i1 = nodes_ind
      coord.append(n1)

    i2 = -1
    for i, node in enumerate(coord):
      if node[0] == n2[0] and node[1] == n2[1]:
        i2 = i
        break
    else:
      nodes_ind += 1
      i2 = nodes_ind
      coord.append(n2)

    i3 = -1
    for i, node in enumerate(coord):
      if node[0] == n3[0] and node[1] == n3[1]:
        i3 = i
        break
    else:
      nodes_ind += 1
      i3 = nodes_ind
      coord.append(n3)

    # Create 4 new triangles
    e1 = [e[0], i1, i3]
    e2 = [i1, e[1], i2]
    e3 = [i3, i2, e[2]]
    e4 = [i2, i3, i1]
    elem.extend([e1, e2, e3, e4])

  # Current level
  lmn = len(emax)

  # Desired level
  lmx = lmn + lvl

  # Loop over new levels
  for l in range(lmn, lmx):
    # create new level
    emax.append([emax[l-1][1]+1, 0])

    # Counter for new elements
    icnt = 0

    # Loop over all elements on last level and create four new elements per old element
    for ie in range(emax[l-1][0], emax[l-1][1]+1):
      elemrefine(ie)
      icnt += 4

    # New last element
    emax[l][1] = emax[l][0] + icnt-1

  t_refine = time.time()-t_refine
  print(f'Refined the mesh for {lvl} more level{"s" if lvl>1 else ""}. The mesh has {emax[-1][1]-emax[-1][0]+1} elements and {len(coord)} nodes now.')


refine(2); print(f'This took {t_refine:.5f}s.')

Create matrix A:

In [ ]:
def area(ie):
  """
  Computes the area of an element based on the formula presented in the lecture.

  Input:
    ie    element index
  """
  # Compute area of element
  e = elem[ie]
  a = np.array(coord[e[1]]) - np.array(coord[e[0]])
  b = np.array(coord[e[2]]) - np.array(coord[e[0]])
  return 0.5 * ( a[0]*b[1] - a[1]*b[0] )

def assmblstiff():
  """
  Sets up the stiffness matrix A like presented in the lecture.

  Input:
    None

  Output:
    A   stiffness matrix
  """
  global t_assemble
  t_assemble = time.time()

  def elstiff(ie):
    # Nodes of this element
    e = elem[ie]

    # Coordinates
    x1, y1 = coord[e[0]]
    x2, y2 = coord[e[1]]
    x3, y3 = coord[e[2]]

    # Denominators
    d1 = (x1-x2)*(y3-y2) - (x3-x2)*(y1-y2)
    d2 = (x2-x3)*(y1-y3) - (x1-x3)*(y2-y3)
    d3 = (x3-x1)*(y2-y1) - (x2-x1)*(y3-y1)

    # Derivatives
    dxb1 = (y3-y2)/d1
    dyb1 = (x3-x2)/d1
    dxb2 = (y1-y3)/d2
    dyb2 = (x1-x3)/d2
    dxb3 = (y2-y1)/d3
    dyb3 = (x2-x1)/d3

    # Build submatrix
    a = np.zeros((3,3))
    ar = area(ie)

    a[0,0] = ar * (dxb1*dxb1 + dyb1*dyb1)
    a[1,1] = ar * (dxb2*dxb2 + dyb2*dyb2)
    a[2,2] = ar * (dxb3*dxb3 + dyb3*dyb3)
    a[1,0] = a[0,1] = ar * (dxb2*dxb1 + dyb2*dyb1)
    a[2,0] = a[0,2] = ar * (dxb3*dxb1 + dyb3*dyb1)
    a[2,1] = a[1,2] = ar * (dxb3*dxb2 + dyb3*dyb2)
    return a

  # Global (sparse) stiffness matrix
  global A
  A = sp.lil_matrix((len(coord),)*2)

  # Loop over all elements and compute the integrals among each other for its three nodes and sum them up in the large matrix A
  l = emax[-1]
  for ie, e in enumerate( elem[ l[0]:l[1]+1 ], start=l[0]):

    # Get the integrals of the nodes of this element among each other
    ae = elstiff(ie)

    # Loop over all combinations of nodes in this element
    for k in range(3):
      i = e[k] # Node index in large matrix

      for l in range(3):
        j = e[l]

        A[i, j] += ae[k, l]

  # Correct the boundary nodes. We have homogenoeous Dirichlet bc, hence 0
  for ie, node in enumerate(coord):
    if (0.0 in node) or (1.0 in node):
      A[ie,:] = A[:,ie] = 0
      A[ie,ie] = 1

  # Change sparse format
  A = A.tocsr()

  t_assemble = time.time()-t_assemble

  print(f"Sparse stiffness matrix A with shape {A.shape} created.")


assmblstiff(); print(f'This took {t_assemble:.5f} seconds.')

Order the matrix with Cuthill-McKee to get a lean structure:

In [ ]:
def reorder_matrix(plot=False):
  """
  Permutes the nodes with the Cuthill-McKee algorithm to slim the sparse matrix.

  Input:
    plot      whether visualize the structures before and after reordering

  Output:
    A         reordered matrix
    perm      permutation array (necessary for the right hand side)
    inv_perm  inverse permutation array (to reconstruct solution u)
  """
  global A, perm, inv_perm

  # Compute ordering
  perm0 = sp.csgraph.reverse_cuthill_mckee(A, symmetric_mode=True)

  # Check if it is already in this ordering
  is_ordered = (np.sum(A!=A[perm0,:][:,perm0])==0)

  if is_ordered:
    print("Matrix already in Cuthill-McKee-ordering.")
  else:
    perm=perm0
    inv_perm = np.argsort(perm)
    Ao = A[perm,:][:,perm]
    print("Computed Cuthill-McKee-ordering.")

  # Inspect structure
  if plot:
    markersize=5 if A.shape[0]<100 else 1

    fig, axes = plt.subplots(1,2-is_ordered, figsize=(8-4*is_ordered,3))
    if is_ordered:
      axes.set_title("Current ordering")
      axes.spy(A, markersize=markersize)
    else:
      axes[0].set_title("Before ordering")
      axes[0].spy(A, markersize=markersize)

      axes[1].set_title("After ordering")
      axes[1].spy(Ao, markersize=markersize)

    plt.show()

  # Adopt ordered matrix
  if not is_ordered:
    A = Ao


reorder_matrix(plot=True)

Define a right-hand-side:

In [ ]:
def f():
  """
  Creates simple initial data.
  """
  return -1*np.ones((len(coord), 1))

def rhs(f):
  """
  Produces the right hand side by multiplying the initial data with the mass matrix.

  Input:
    f   initial data

  Output:
    b   right hand side M*f
  """
  global t_rhs
  t_rhs = time.time()

  b = np.zeros(len(f))

  l = emax[-1]
  for ie, e in enumerate( elem[l[0]:l[1]+1], start=l[0] ):
    # Surface area times mean f values on the nodes
    b[e] += area(ie) * np.sum(f[e]) / 3

  # Boundary values
  for i, node in enumerate(coord):
    if (0.0 in node) or (1.0 in node):
      b[i] = 0
  t_rhs = time.time()-t_rhs

  print("Created the right hand side.")
  return b[perm]

b = rhs(f()); print(f'Creating the rhs took {t_rhs:0.5f}s')

Solve:

In [ ]:
def solve(b):
  """
  Solves the problem for the given rhs.

  Input:
    b   right-hand side of the problem

  Output:
    u   solution to the PDE
  """
  global t_solve, A
  t_solve = time.time()
  u = spsolve(A, b)[inv_perm]
  t_solve = time.time()-t_solve

  print("System solved.")
  return u

u = solve(b); print(f'Solving took {t_solve:.5f}s.')

In [ ]:
def plot_solution(u):
  """
  Plots the solution.

  Input:
    u   solution to the FEM
  """

  print("=============================================")
  print("===========       STATISTICS      ===========")
  print("=============================================")
  print(f"| INFO: time for refine:            {t_refine:.5f}s|")
  print(f"| INFO: time for assemble:          {t_assemble:.5f}s|")
  print(f"| INFO: time for right-hand-side:   {t_rhs:.5f}s|")
  print(f"| INFO: time for solve:             {t_solve:.5f}s|")
  print(f"| INFO: Number of nodes:            {len(coord):8d}|")
  print(f"| INFO: Number of elements:         {emax[-1][1]-emax[-1][0]+1:8d}|")
  print("=============================================")
  grid = np.array(coord).T
  fig = plt.figure()
  ax = fig.add_subplot(111, projection='3d')
  ax.plot_trisurf(grid[0], grid[1], u)
  ax.set_xlabel('X')
  ax.set_ylabel('Y')
  ax.set_zlabel('Z')
  plt.show()

plot_solution(u)

All in one:

In [ ]:
initial()
refine(4)
assmblstiff()
reorder_matrix(plot=False)
u = solve(rhs(f()))
plot_solution(u)